## Acquisitor and Cleaner

In [1]:
from marvin_python_toolbox.common.data import MarvinData
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

initial_dataset = pd.read_csv(
    MarvinData.download_file("https://s3.amazonaws.com/automl-example/produtos.csv"),
    delimiter=";", encoding='utf-8')


nltk.download('stopwords')
stop_words = stopwords.words('portuguese')

def remove_stop_words(line):
    return ' '.join([w for w in word_tokenize(line) if not w in stop_words])

initial_dataset["text"] = initial_dataset["nome"] + " " + initial_dataset["descricao"]
initial_dataset.drop(["descricao", "nome"], axis=1, inplace=True)
initial_dataset.dropna(inplace=True)
initial_dataset['text'] = initial_dataset['text'].apply(remove_stop_words)

marvin_initial_dataset = initial_dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vinicius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Training preparator

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    marvin_initial_dataset["text"], 
    marvin_initial_dataset["categoria"], 
    test_size = 0.2, 
    random_state = 10
)

vect = CountVectorizer()
vect.fit(marvin_initial_dataset["text"])

marvin_dataset = {
    "X_train": vect.transform(X_train),
    "X_test": vect.transform(X_test),
    "y_train": y_train,
    "y_test" : y_test,
    "vect": vect
}

## Training 

In [3]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(marvin_dataset["X_train"], marvin_dataset["y_train"])

marvin_model = {
    "clf" : clf,
    "vect": marvin_dataset["vect"]
}

## Evaluation

In [4]:
from sklearn.metrics import classification_report


y_prediction = marvin_model["clf"].predict(marvin_dataset["X_test"])

report = classification_report(y_prediction, marvin_dataset["y_test"])

print(report)

marvin_metrics = report

             precision    recall  f1-score   support

  brinquedo       0.98      0.99      0.98       140
       game       0.97      0.96      0.96       126
      livro       0.97      0.98      0.98       156
  maquiagem       1.00      0.99      1.00       162

avg / total       0.98      0.98      0.98       584



## Predictor preparation

In [6]:
input_message = {"text": """God of War"""}

In [7]:
input_message = marvin_model["vect"].transform([input_message["text"]])

## Predictor

In [8]:
final_prediction = marvin_model["clf"].predict(input_message)[0]

In [9]:
print(final_prediction)

game
